In [1]:
import warnings
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder
%run models.ipynb

warnings.filterwarnings('ignore')
random.seed(100)

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [52]:
dataset = "docred"

In [53]:
features = pd.read_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Features/features_"+dataset+".csv", index_col=0)
# features.head(1)

In [54]:
features["rel type"].unique()

array(['performer', 'member of', 'child', 'inception', 'record label',
       'has part', 'date of birth',
       'located in the administrative territorial entity',
       'date of death', 'place of birth', 'country', 'award received',
       'sibling', 'country of citizenship',
       'contains administrative territorial entity', 'league',
       'publication date', 'capital', 'location', 'employer',
       'headquarters location', 'parent taxon', 'educated at',
       'work location', 'creator', 'start time', 'place of death',
       'cast member', 'head of government', 'chairperson',
       'mouth of the watercourse', 'head of state',
       'member of sports team', 'part of', 'spouse', 'point in time',
       'followed by', 'characters', 'present in work', 'father', 'mother',
       'director', 'manufacturer', 'subsidiary', 'owned by',
       'parent organization', 'member of political party', 'subclass of',
       'conflict', 'notable work', 'genre', 'screenwriter', 'continent',


In [55]:
column_names = ['sent_index', 'sentence_length', 'root_word', 'root_index',
       'entity_1_index', 'entity_2_index', 'entity_distance',
       'no_words_before_entity_1', 'no_words_after_entity_2',
       'entity_1_root_distance', 'entity_2_root_distance', 'entity 1 name',
       'entity 2 name', 'entity_type_1', 'entity_type_2', 'entity_pos_1',
       'entity_pos_2', 'entity_dep_1', 'entity_dep_2', 'entity_tag_1',
       'entity_tag_2', 'shortest_distance', 'rel type']
features = features[column_names]
features.rename(columns={"rel type": "label"}, inplace=True)

In [56]:
X = features.iloc[:,:-1]
X = X.fillna("")
y_original =  features.iloc[:,-1]

x_train_original, x_test_original, y_train_original, y_test_original = train_test_split(X, y_original, test_size=0.2)
onehot_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
x_train = onehot_encoder.fit_transform(x_train_original)
x_test = onehot_encoder.transform(x_test_original)

le = LabelEncoder()
y_train = le.fit_transform(y_train_original)
y_test = le.transform(y_test_original)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping

{'applies to jurisdiction': 0,
 'author': 1,
 'award received': 2,
 'basin country': 3,
 'capital': 4,
 'capital of': 5,
 'cast member': 6,
 'chairperson': 7,
 'characters': 8,
 'child': 9,
 'composer': 10,
 'conflict': 11,
 'contains administrative territorial entity': 12,
 'continent': 13,
 'country': 14,
 'country of citizenship': 15,
 'country of origin': 16,
 'creator': 17,
 'date of birth': 18,
 'date of death': 19,
 'developer': 20,
 'director': 21,
 'dissolved, abolished or demolished': 22,
 'educated at': 23,
 'employer': 24,
 'end time': 25,
 'ethnic group': 26,
 'father': 27,
 'followed by': 28,
 'follows': 29,
 'founded by': 30,
 'genre': 31,
 'has part': 32,
 'head of government': 33,
 'head of state': 34,
 'headquarters location': 35,
 'inception': 36,
 'influenced by': 37,
 'instance of': 38,
 'languages spoken, written or signed': 39,
 'league': 40,
 'legislative body': 41,
 'located in or next to body of water': 42,
 'located in the administrative territorial entity': 

In [36]:
# unique, count = np.unique(y_original, return_counts = True)
# dict(zip(unique, count))

In [8]:
pred = logistic_regression_multiple_classes(x_train, x_test, y_train, y_test, "et")
x_test_log_pred = x_test_original
x_test_log_pred["label"] = y_test
x_test_log_pred["prediction"] = pred
x_test_log_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_logisitic_regression_prediction.csv")

--------Logistic Regression-----------


FileNotFoundError: [Errno 2] No such file or directory: '/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Models/et_lr_model_ACE2004.sav'

In [96]:
pred = random_forest_multiple(x_train, x_test, y_train, y_test,"et")
x_test_rf_pred = x_test_original
x_test_rf_pred["label"] = y_test
x_test_rf_pred["prediction"] = pred
x_test_rf_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_random_forest_prediction.csv")


--------Random Forest-----------


In [97]:
pred = svm_multiple("linear", x_train, x_test, y_train, y_test, "et")
x_test_svm_pred = x_test_original
x_test_svm_pred["label"] = y_test
x_test_svm_pred["prediction"] = pred
x_test_svm_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_prediction.csv")

--------Support Vector Classifier linear-----------


In [98]:
pred = svm_multiple("rbf", x_train, x_test, y_train, y_test, "et")
x_test_svmrbf_pred = x_test_original
x_test_svmrbf_pred["label"] = y_test
x_test_svmrbf_pred["prediction"] = pred
x_test_svmrbf_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_rbf_prediction.csv")

--------Support Vector Classifier rbf-----------


In [99]:
pred = svm_multiple("poly", x_train, x_test, y_train, y_test, "et")
x_test_svmpoly_pred = x_test_original
x_test_svmpoly_pred["label"] = y_test
x_test_svmpoly_pred["prediction"] = pred
x_test_svmpoly_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_poly_prediction.csv")


--------Support Vector Classifier poly-----------


In [100]:
pred = xgboost_multiple_class(x_train, x_test, y_train, y_test, "et")
x_test_xg_pred = x_test_original
x_test_xg_pred["label"] = y_test
x_test_xg_pred["prediction"] = pred
x_test_xg_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_xgboost_prediction.csv")


In [101]:
pred = adaboost_multiple(x_train, x_test, y_train, y_test, "et")
x_test_ada_pred = x_test_original
x_test_ada_pred["label"] = y_test
x_test_ada_pred["prediction"] = pred
x_test_ada_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_adaboost_prediction.csv")
